In [2]:
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from category_encoders import OrdinalEncoder
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from scipy.stats.distributions import uniform
from sklearn.model_selection import RandomizedSearchCV

In [3]:
def multiple_load_dataset():
    with open('/Users/yerin/AIB/project/TP1/dataset/mulit_classification_data.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        rows = []
        for row in csvreader:
            rows.append(row)

    data = rows
    #추가한 부분
    return data

data = multiple_load_dataset()


def combine_labels(data):
    labels = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']
    one_hot_indices = [27, 28, 29, 30, 31, 32, 33]

    new_data = []

    for row in data:
        independent_vars = row[:27]  # Extract the first 27 columns as the independent variables
        row_labels = []

        for i, label in enumerate(labels):
            if row[one_hot_indices[i]] == '1':
                row_labels.append(label)

        new_row = independent_vars + row_labels
        new_data.append(new_row)

    return new_data

new_data = combine_labels(data)

In [4]:
X = [row[:26] for row in new_data]
y = [row[27] for row in new_data]

In [5]:
def standard_scaler(X, y):
    
    encoder = OrdinalEncoder()
    y = encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state=42)

    scaler = StandardScaler()
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = standard_scaler(X, y)

In [17]:
model = XGBClassifier(objective = "binary:logistic",
                      n_estimators = 140,
                      max_depth = 30, 
                      random_state=42,
                      gamma=1,
                      learning_rate = 0.1)

ovr = OneVsRestClassifier(model)
ovr.fit(X_train, y_train)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=1,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=30,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=140, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=42, ...))

In [18]:
from sklearn.metrics import accuracy_score
y_val_pred = ovr.predict(X_val)
y_test_pred = ovr.predict(X_test)

print("Validation Set accuracy score")
print(accuracy_score(y_val, y_val_pred))
print('==============================')
print("test Set accuracy score")
print(accuracy_score(y_test, y_test_pred))

Validation Set accuracy score
0.7427652733118971
test Set accuracy score
0.7789203084832905


In [19]:
ovo = OneVsOneClassifier(model)

ovo.fit(X_train, y_train)
y_val_pred = ovo.predict(X_val)
y_test_pred = ovo.predict(X_test)

/Users/yerin/opt/anaconda3/envs/tp1/lib/python3.8/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
print("Validation Set accuracy score")
print(accuracy_score(y_val, y_val_pred))
print('==============================')
print("test Set accuracy score")
print(accuracy_score(y_test, y_test_pred))

Validation Set accuracy score
0.7459807073954984
test Set accuracy score
0.7840616966580977


In [21]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_val, y_val_pred)
confusion_matrix

array([[14,  0,  0,  0,  0,  2, 12],
       [ 0, 18,  0,  0,  0,  0,  5],
       [ 1,  0, 55,  1,  0,  1,  2],
       [ 0,  0,  0, 10,  0,  0,  0],
       [ 1,  0,  0,  0, 10,  0,  1],
       [ 2,  0,  0,  0,  0, 42, 29],
       [ 9,  0,  5,  0,  0,  8, 83]])